In [1]:
import requests
import pandas as pd
from binance.client import Client
from secrets import secrets

auth = requests.auth.HTTPBasicAuth(secrets['REDDIT_API_KEY'], secrets['REDDIT_SECRET_KEY'])
headers = {'User-Agent': 'MyBot/0.0.1'}

data = {'grant_type': 'password', 'username': secrets['REDDIT_USERNAME'], 'password': secrets['REDDIT_PASSWORD']}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [2]:
BINANCE_API_KEY = secrets['BINANCE_API_KEY']
BINANCE_SECRET_KEY = secrets['BINANCE_SECRET_KEY']
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
info = client.get_exchange_info()
coins = [s['baseAsset'] for s in info['symbols']]
coins = list(set(coins))
len(coins)

382

In [3]:
df = pd.DataFrame(columns=['coin', 'mentions'])
df['coin'] = coins
df['mentions'] = 0
df.head()

,coin,mentions
0,FIS,0
1,BCD,0
2,VIA,0
3,DF,0
4,MDT,0


In [6]:
res = requests.get("https://oauth.reddit.com/r/cryptocurrency/new", headers=headers, params={'limit': 50})

for post in res.json()['data']['children']:
    text = post['data']['title'] + ' ' + post['data']['selftext']
    res2 = requests.get("https://oauth.reddit.com/r/cryptocurrency/comments/" + post['data']['id'],
                        headers=headers)
    for p_comment in res2.json():
        for c_comment in p_comment['data']['children']:
            if 'body' in c_comment['data'].keys():
                text = text + ' ' + c_comment['data']['body']
    for coin in coins:
        if coin in text:
            df.loc[df['coin'] == coin, 'mentions'] += 1

df = df.sort_values(by='mentions', ascending=False)
df.head(10)

,coin,mentions
188,ETH,15
214,BTC,14
189,ADA,7
37,ENJ,6
285,USDC,6
160,LTC,6
173,BAT,4
96,LINK,4
358,PAX,3
116,PAXG,3


In [5]:
res = requests.get("https://oauth.reddit.com/r/cryptocurrency/hot", headers=headers, params={'limit': 100})

for post in res.json()['data']['children']:
    text = post['data']['title'] + ' ' + post['data']['selftext']
    for coin in coins:
        if coin in text:
            df.loc[df['coin'] == coin, 'mentions'] += 1

df = df.sort_values(by='mentions', ascending=False)
df.head(10)

,coin,mentions
121,ETH,19
290,BTC,18
269,ADA,8
313,NEO,7
75,BAT,7
355,LTC,6
326,XLM,5
333,BCH,5
110,DOT,4
367,IOTA,4


In [8]:
df.head(10).to_dict('records')

[{'coin': 'ETH', 'mentions': 19},
 {'coin': 'BTC', 'mentions': 18},
 {'coin': 'ADA', 'mentions': 8},
 {'coin': 'NEO', 'mentions': 7},
 {'coin': 'BAT', 'mentions': 7},
 {'coin': 'LTC', 'mentions': 6},
 {'coin': 'XLM', 'mentions': 5},
 {'coin': 'BCH', 'mentions': 5},
 {'coin': 'DOT', 'mentions': 4},
 {'coin': 'IOTA', 'mentions': 4}]

In [11]:
def get_reddit_trending(coins, sub='cryptocurrency', mode='hot'):
    auth = requests.auth.HTTPBasicAuth(secrets['REDDIT_API_KEY'], secrets['REDDIT_SECRET_KEY'])
    headers = {'User-Agent': 'MyBot/0.0.1'}
    data = {'grant_type': 'password', 'username': secrets['REDDIT_USERNAME'], 'password': secrets['REDDIT_PASSWORD']}
    res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
    TOKEN = res.json()['access_token']
    headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
    df = pd.DataFrame(columns=['coin', 'mentions'])
    df['coin'] = coins
    df['mentions'] = 0
    res = requests.get('https://oauth.reddit.com/r/{}/{}'.format(sub, mode), headers=headers, params={'limit': 100})
    for post in res.json()['data']['children']:
        text = post['data']['title'] + ' ' + post['data']['selftext']
        for coin in coins:
            if coin in text:
                df.loc[df['coin'] == coin, 'mentions'] += 1
    df = df.sort_values(by='mentions', ascending=False)
    return df.head(10).to_dict('records')

In [12]:
get_reddit_trending(coins)

[{'coin': 'BTC', 'mentions': 12},
 {'coin': 'ETH', 'mentions': 11},
 {'coin': 'NEO', 'mentions': 5},
 {'coin': 'BAT', 'mentions': 4},
 {'coin': 'ADA', 'mentions': 4},
 {'coin': 'LTC', 'mentions': 4},
 {'coin': 'BCH', 'mentions': 3},
 {'coin': 'XLM', 'mentions': 3},
 {'coin': 'ALGO', 'mentions': 2},
 {'coin': 'GO', 'mentions': 2}]